https://huggingface.co/docs/transformers/en/tasks/asr

In [ ]:
!pip install librosa soundfile

In [2]:
from datasets import load_dataset, Audio, DatasetDict, concatenate_datasets
import pandas as pd
import json

In [5]:
lang_used=['zh-CN', 'ru-RU', 'fr-FR', 'en-US', 'de-DE']
minds=[]
for i in lang_used:
    minds_data = load_dataset("PolyAI/minds14", i, trust_remote_code=True)
    minds.append(minds_data['train'])

minds = concatenate_datasets(minds)
minds = minds.train_test_split(test_size=0.4, seed = 21, stratify_by_column="intent_class")
minds_test = minds['test'].train_test_split(test_size=0.5)
dataset_dict = DatasetDict({
    'train': minds['train'],
    'test': minds_test['test'],
    'valid': minds_test['train']})

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
dataset_dict["train"].shape,dataset_dict["test"].shape,dataset_dict["valid"].shape

((1652, 6), (551, 6), (551, 6))

In [10]:
dataset_dict["train"][0].keys()

dict_keys(['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'])

In [11]:
dataset_dict["train"][0]['path']

'C:\\Users\\andre\\.cache\\huggingface\\datasets\\downloads\\extracted\\482a593b1882116f1ef934ce05f74608ebffded5fe81b88e49a3d63337574cf4\\de-DE~CARD_ISSUES\\CA3ee78582d721d7ac7c06a3dd33b035f8_0.wav'

In [12]:
dataset_dict["train"][0]['audio']

{'path': 'C:\\Users\\andre\\.cache\\huggingface\\datasets\\downloads\\extracted\\482a593b1882116f1ef934ce05f74608ebffded5fe81b88e49a3d63337574cf4\\de-DE~CARD_ISSUES\\CA3ee78582d721d7ac7c06a3dd33b035f8_0.wav',
 'array': array([0.00024414, 0.00048828, 0.00048828, ..., 0.00024414, 0.00024414,
        0.00024414]),
 'sampling_rate': 8000}

In [13]:
intent = dataset_dict["train"].features["intent_class"]

In [14]:
intent.names

['abroad',
 'address',
 'app_error',
 'atm_limit',
 'balance',
 'business_loan',
 'card_issues',
 'cash_deposit',
 'direct_debit',
 'freeze',
 'high_value_payment',
 'joint_account',
 'latest_transactions',
 'pay_bill']

In [15]:
lang = dataset_dict["train"].features["lang_id"]

In [16]:
lang.names

['cs-CZ',
 'de-DE',
 'en-AU',
 'en-GB',
 'en-US',
 'es-ES',
 'fr-FR',
 'it-IT',
 'ko-KR',
 'nl-NL',
 'pl-PL',
 'pt-PT',
 'ru-RU',
 'zh-CN']

In [17]:
df = pd.DataFrame(dataset_dict["train"])

In [18]:
df["intent_label"] = df["intent_class"].apply(lambda x: intent.int2str(x))

In [19]:
df["lang"] = df["lang_id"].apply(lambda x: lang.int2str(x))

In [20]:
def audio_array_len(x):
    return len(x["array"])
df["array_len"] = df["audio"].apply(lambda x: audio_array_len(x))

In [21]:
df.shape

(1652, 9)

In [22]:
df.head()

,path,audio,transcription,english_transcription,intent_class,lang_id,intent_label,lang,array_len
0,C:\Users\andre\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\andre\.cache\huggingface\da...,Hallo kurze frage ich wollte nur wissen warum ...,Hello short question I just wanted to know why...,6,1,card_issues,de-DE,106880
1,C:\Users\andre\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\andre\.cache\huggingface\da...,hi I am traveling in Italy right now and I'm j...,hi I am traveling in Italy right now and I'm j...,0,4,abroad,en-US,73561
2,C:\Users\andre\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\andre\.cache\huggingface\da...,nein noch nicht,no not yet,0,1,abroad,de-DE,29600
3,C:\Users\andre\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\andre\.cache\huggingface\da...,Почему моя карта не работает,Why is my card not working,6,12,card_issues,ru-RU,24520
4,C:\Users\andre\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\andre\.cache\huggingface\da...,я получила эсэмэску потому что пытаюсь заплати...,I received a text message because I am trying ...,10,12,high_value_payment,ru-RU,59443


In [23]:
df.to_csv("minds14-all-train.csv", index=False)
df.to_csv("minds14-all-test.csv", index=False)
df.to_csv("minds14-all-valid.csv", index=False)